In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stat
from openpyxl import load_workbook
from scipy.optimize import fsolve
import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
# Loading and cleaning the data
def process(excel):
    df = pd.read_excel('frequency_analysis.xlsx', sheet_name = excel)
    df.sort_values(by = 'Rainfall', inplace = True, ascending = False)
    df.reset_index(inplace = True)
    df.reset_index(inplace = True)
    df.rename(columns = {'level_0': 'Rank', 'Rainfall': 'RR (mm)', 'Year': 'Tahun'}, inplace = True)
    df['Rank'] += 1
    df.drop(['index'], axis = 1, inplace = True)
    df['Log'] = np.log10(df['RR (mm)'])
    return df

# Calculating the statistical parameters
def rain(df):
    rain_stat = {}
    rain_stat['mean'] = np.mean(df['RR (mm)'])
    rain_stat['std'] = stat.tstd(df['RR (mm)'])
    rain_stat['skew'] = stat.skew(df['RR (mm)'], bias = False)
    return rain_stat

def log(df):
    log_stat = {}
    log_stat['mean'] = np.mean(df['Log'])
    log_stat['std'] = stat.tstd(df['Log'])
    log_stat['skew'] = stat.skew(df['Log'], bias = False)
    return log_stat

# Normal distribution
def normal(df, stat_param):
    normal_df = df[['Rank', 'RR (mm)']]
    normal_df['P_emp'] = normal_df['Rank']/(len(normal_df) + 1)
    normal_df['k'] = (normal_df['RR (mm)'] - stat_param['mean'])/stat_param['std']
    normal_df['P_teoritis'] = 1 - stat.norm.cdf(normal_df['k'])
    normal_df['Delta P'] = abs(normal_df['P_teoritis'] - normal_df['P_emp'])
    return normal_df

# Log-Normal distribution
def lognormal(df, stat_param):
    lognormal_df = df[['Rank', 'Log']]
    lognormal_df['P_emp'] = lognormal_df['Rank']/(len(lognormal_df) + 1)
    lognormal_df['k'] = (lognormal_df['Log'] - stat_param['mean'])/stat_param['std']
    lognormal_df['P_teoritis'] = 1 - stat.norm.cdf(lognormal_df['k'])
    lognormal_df['Delta P'] = abs(lognormal_df['P_teoritis'] - lognormal_df['P_emp'])
    return lognormal_df

# Gumbel distribution
def gumbel(df, stat_param):
    gumbel_df = df[['Rank', 'RR (mm)']]
    gumbel_df['P_emp'] = gumbel_df['Rank']/(len(gumbel_df) + 1)
    gumbel_df['k'] = (gumbel_df['RR (mm)'] - stat_param['mean'])/stat_param['std']
    gumbel_df['P_teoritis'] = 1/(np.exp(np.exp(-np.pi/np.sqrt(6)*gumbel_df['k']-.5772))/(np.exp(np.exp(-np.pi/np.sqrt(6)*gumbel_df['k']-.5772))-1))
    gumbel_df['Delta P'] = abs(gumbel_df['P_teoritis'] - gumbel_df['P_emp'])
    return gumbel_df

# Log-Pearson 3 distribution
def lp3(df, stat_param):
    lp3_df = df[['Rank', 'Log']]
    lp3_df['P_emp'] = lp3_df['Rank']/(len(lp3_df) + 1)
    lp3_df['k'] = (lp3_df['Log'] - stat_param['mean'])/stat_param['std']
    k = stat_param['skew']/6
    P_teoritis = []
    for KT in lp3_df['k']:
        def eq(z):
            return z + (z**2-1)*k + 1/3*(z**3-6*z)*k**2 - (z**2-1)*k**3 + z*k**4 + 1/3*k**5 - KT
        P_teoritis.append(1-stat.norm.cdf(fsolve(eq, .1)[0]))
    lp3_df['P_teoritis'] = P_teoritis
    lp3_df['Delta P'] = abs(lp3_df['P_teoritis'] - lp3_df['P_emp'])
    return lp3_df

# Choosing the distribution that fits the data
def critical_ks(n):
    upper = np.ceil(n/5)*5
    lower = np.floor(n/5)*5
    if n > 10 and n < 15: critical = .41 + (n-lower)*(.34-.41)/(upper-lower)
    elif n == 10: critical = .41
    elif n > 15 and n < 20: critical = .34 + (n-lower)*(.29-.34)/(upper-lower)
    elif n == 15: critical = .34
    return critical

def ks_test(normal, log, gumbel, lp3):
    normal_ks = max(normal['Delta P'])
    log_ks = max(log['Delta P'])
    gumbel_ks = max(gumbel['Delta P'])
    lp3_ks = max(lp3['Delta P'])
    length = len(normal)
    ks_result = pd.DataFrame([
        ['Normal', normal_ks, critical_ks(length),],
        ['Log Normal', log_ks, critical_ks(length),],
        ['Gumbel', gumbel_ks, critical_ks(length)],
        ['Log Pearson 3', lp3_ks, critical_ks(length)]
    ], columns = ['Distribusi', 'Delta P', 'Nilai Kritis'])
    ks_result['Kesimpulan'] = np.where(ks_result['Delta P'] < ks_result['Nilai Kritis'], 'Diterima', 'Ditolak')
    print('Use', ks_result.loc[ks_result['Delta P'] == min(ks_result['Delta P']), 'Distribusi'].values[0])
    return ks_result

# Calculating the rainfall design for various return periods
def design(stat_param, dist):
    if dist == 'lp3':
        T = [2,5,10,25,50,100,200,1000]
        Z = [-stat.norm.ppf(1/t, 0, 1) for t in T]
        rf_design = pd.DataFrame({'T (Tahun)': T, 'z': Z})
        k = stat_param['skew']/6
        rf_design['K'] = rf_design['z'] + (rf_design['z']**2-1)*k + 1/3*(rf_design['z']**3-6*rf_design['z'])*k**2 - (rf_design['z']**2-1)*k**3 + rf_design['z']*k**4 + 1/3*k**5
        rf_design['Rt (mm)'] = 10**(stat_param['mean'] + rf_design['K']*stat_param['std'])
        return rf_design
    
    elif dist == 'gumbel':
        T = [2,5,10,25,50,100,200,1000]
        rf_design = pd.DataFrame({'T (Tahun)': T})
        rf_design['K'] = -(6**.5)/np.pi*(.5772 + np.log(np.log(rf_design['T (Tahun)']/(rf_design['T (Tahun)']-1))))
        rf_design['Rt (mm)'] = stat_param['mean'] + rf_design['K']*stat_param['std']
        return rf_design
    
    elif dist == 'normal':
        T = [2,5,10,25,50,100,200,1000]
        K = [-stat.norm.ppf(1/t, 0, 1) for t in T]
        rf_design = pd.DataFrame({'T (Tahun)': T, 'K': K})
        rf_design['Rt (mm)'] = stat_param['mean'] + rf_design['K']*stat_param['std']
        return rf_design

# Saving results to Excel for structured storage
def save_excel(dfs, sheet):
    dfs[1] = pd.DataFrame.from_dict(data = dfs[1], orient='index').reset_index()
    dfs[2] = pd.DataFrame.from_dict(data = dfs[2], orient='index')
    start = len(dfs[0])
    positions = [
        (0,0), (start+1, 1), (start+1, 3),
        (start+5, 0), (start+5, 7), (start*2+7, 0), (start*2+7, 7),
        (0, 14), (7, 14)
    ]
    
    wb = load_workbook('frequency_summary.xlsx')
    if sheet in wb.sheetnames:
        std = wb[sheet]
        wb.remove(std)
    wb.create_sheet(sheet)
    wb.save('frequency_summary.xlsx')
    wb.close()  
    
    with pd.ExcelWriter(
        'frequency_summary.xlsx',
        engine = 'openpyxl',
        mode = 'a',
        if_sheet_exists = 'overlay'
    ) as writer:
        for df, count, (row, col) in zip(dfs, range(len(dfs)), positions):
            index, header = False, True
            if count == 1 or count == 2: header = False
            if count == 1: index = True 
            df.to_excel(writer, sheet_name = sheet, startrow = row, startcol = col, index = False, header = header)

pd.options.mode.chained_assignment = None  # Suppress SettingWithCopyWarning

# Frequency Analysis

## Sunter Watershed

In [111]:
halim_df = process('halim')
halim_rain = rain(halim_df)
halim_log = log(halim_df)

In [112]:
halim_normal = normal(halim_df, halim_rain)
halim_normal

,Rank,RR (mm),P_emp,k,P_teoritis,Delta P
0,1,305,0.090909,2.545079,0.005463,0.085446
1,2,161,0.181818,0.822035,0.205528,0.023710
2,3,95,0.272727,0.032307,0.487114,0.214386
3,4,72,0.363636,-0.242901,0.595959,0.232323
4,5,63,0.454545,-0.350591,0.637053,0.182507
5,6,58,0.545455,-0.410419,0.659251,0.113796
6,7,54,0.636364,-0.458282,0.676625,0.040261
7,8,47,0.727273,-0.542041,0.706105,0.021168
8,9,42,0.818182,-0.601869,0.726369,0.091813
9,10,26,0.909091,-0.793318,0.786204,0.122887


In [113]:
halim_lognormal = lognormal(halim_df, halim_log)
halim_lognormal

,Rank,Log,P_emp,k,P_teoritis,Delta P
0,1,2.484300,0.090909,2.063251,0.019544,0.071365
1,2,2.206826,0.181818,1.156155,0.123809,0.058009
2,3,1.977724,0.272727,0.407192,0.341933,0.069206
3,4,1.857332,0.363636,0.013619,0.494567,0.130931
4,5,1.799341,0.454545,-0.175963,0.569839,0.115293
5,6,1.763428,0.545455,-0.293366,0.615379,0.069924
6,7,1.732394,0.636364,-0.394820,0.653512,0.017149
7,8,1.672098,0.727273,-0.591935,0.723053,0.004220
8,9,1.623249,0.818182,-0.751627,0.773862,0.044320
9,10,1.414973,0.909091,-1.432506,0.924000,0.014910


In [114]:
halim_gumbel = gumbel(halim_df, halim_rain)
halim_gumbel

,Rank,RR (mm),P_emp,k,P_teoritis,Delta P
0,1,305,0.090909,2.545079,0.021235,0.069674
1,2,161,0.181818,0.822035,0.177688,0.004130
2,3,95,0.272727,0.032307,0.416481,0.143754
3,4,72,0.363636,-0.242901,0.535454,0.171817
4,5,63,0.454545,-0.350591,0.585321,0.130775
5,6,58,0.545455,-0.410419,0.613434,0.067980
6,7,54,0.636364,-0.458282,0.636009,0.000355
7,8,47,0.727273,-0.542041,0.675425,0.051848
8,9,42,0.818182,-0.601869,0.703284,0.114898
9,10,26,0.909091,-0.793318,0.788414,0.120676


In [115]:
halim_lp3 = lp3(halim_df, halim_log)
halim_lp3

,Rank,Log,P_emp,k,P_teoritis,Delta P
0,1,2.484300,0.090909,2.063251,0.037891,0.053018
1,2,2.206826,0.181818,1.156155,0.125079,0.056739
2,3,1.977724,0.272727,0.407192,0.294748,0.022020
3,4,1.857332,0.363636,0.013619,0.432753,0.069116
4,5,1.799341,0.454545,-0.175963,0.510071,0.055525
5,6,1.763428,0.545455,-0.293366,0.560458,0.015004
6,7,1.732394,0.636364,-0.394820,0.604953,0.031410
7,8,1.672098,0.727273,-0.591935,0.691857,0.035416
8,9,1.623249,0.818182,-0.751627,0.760080,0.058102
9,10,1.414973,0.909091,-1.432506,0.966044,0.056953


In [116]:
halim_test = ks_test(halim_normal, halim_lognormal, halim_gumbel, halim_lp3)
halim_test

Use Log Pearson 3


,Distribusi,Delta P,Nilai Kritis,Kesimpulan
0,Normal,0.232323,0.41,Diterima
1,Log Normal,0.130931,0.41,Diterima
2,Gumbel,0.171817,0.41,Diterima
3,Log Pearson 3,0.069116,0.41,Diterima


In [117]:
halim_design = design(halim_log, 'lp3')
halim_design

,T (Tahun),z,K,Rt (mm)
0,2,-0.000000,-0.152025,64.071219
1,5,0.841621,0.761744,121.949028
2,10,1.281552,1.334806,182.588687
3,25,1.750686,2.024132,296.711111
4,50,2.053749,2.514660,419.162864
5,100,2.326348,2.987530,584.830889
6,200,2.575829,3.447439,808.561278
7,1000,3.090232,4.480893,1674.322101


In [118]:
save_excel(
    [
        halim_df, halim_rain, halim_log, halim_normal, halim_lognormal,
        halim_gumbel, halim_lp3, halim_test, halim_design
    ], 'halim_design'
)

## Angke Watershed

In [119]:
banten_df = process('banten')
banten_rain = rain(banten_df)
banten_log = log(banten_df)

In [120]:
banten_normal = normal(banten_df, banten_rain)
banten_normal

,Rank,RR (mm),P_emp,k,P_teoritis,Delta P
0,1,282.6,0.066667,3.143744,0.000834,0.065833
1,2,123.8,0.133333,0.403430,0.343316,0.209983
2,3,117.0,0.200000,0.286086,0.387406,0.187406
3,4,108.5,0.266667,0.139407,0.444564,0.177898
4,5,105.0,0.333333,0.079010,0.468512,0.135179
5,6,97.0,0.400000,-0.059041,0.523540,0.123540
6,7,96.0,0.466667,-0.076298,0.530409,0.063742
7,8,84.0,0.533333,-0.283375,0.611555,0.078222
8,9,76.0,0.600000,-0.421426,0.663278,0.063278
9,10,76.0,0.666667,-0.421426,0.663278,0.003389


In [121]:
banten_lognormal = lognormal(banten_df, banten_log)
banten_lognormal

,Rank,Log,P_emp,k,P_teoritis,Delta P
0,1,2.451172,0.066667,2.307204,0.010522,0.056145
1,2,2.092721,0.133333,0.655777,0.255984,0.122650
2,3,2.068186,0.200000,0.542743,0.293654,0.093654
3,4,2.035430,0.266667,0.391831,0.347591,0.080925
4,5,2.021189,0.333333,0.326224,0.372127,0.038794
5,6,1.986772,0.400000,0.167658,0.433426,0.033426
6,7,1.982271,0.466667,0.146924,0.441596,0.025071
7,8,1.924279,0.533333,-0.120251,0.547858,0.014525
8,9,1.880814,0.600000,-0.320503,0.625706,0.025706
9,10,1.880814,0.666667,-0.320503,0.625706,0.040960


In [122]:
banten_gumbel = gumbel(banten_df, banten_rain)
banten_gumbel

,Rank,RR (mm),P_emp,k,P_teoritis,Delta P
0,1,282.6,0.066667,3.143744,0.009910,0.056756
1,2,123.8,0.133333,0.403430,0.284424,0.151090
2,3,117.0,0.200000,0.286086,0.322281,0.122281
3,4,108.5,0.266667,0.139407,0.374712,0.108045
4,5,105.0,0.333333,0.079010,0.397917,0.064584
5,6,97.0,0.400000,-0.059041,0.454273,0.054273
6,7,96.0,0.466667,-0.076298,0.461619,0.005047
7,8,84.0,0.533333,-0.283375,0.554048,0.020715
8,9,76.0,0.600000,-0.421426,0.618622,0.018622
9,10,76.0,0.666667,-0.421426,0.618622,0.048045


In [123]:
banten_lp3 = lp3(banten_df, banten_log)
banten_lp3

,Rank,Log,P_emp,k,P_teoritis,Delta P
0,1,2.451172,0.066667,2.307204,0.008196,0.058471
1,2,2.092721,0.133333,0.655777,0.259514,0.126180
2,3,2.068186,0.200000,0.542743,0.298372,0.098372
3,4,2.035430,0.266667,0.391831,0.353696,0.087029
4,5,2.021189,0.333333,0.326224,0.378729,0.045396
5,6,1.986772,0.400000,0.167658,0.440892,0.040892
6,7,1.982271,0.466667,0.146924,0.449136,0.017531
7,8,1.924279,0.533333,-0.120251,0.555462,0.022128
8,9,1.880814,0.600000,-0.320503,0.632327,0.032327
9,10,1.880814,0.666667,-0.320503,0.632327,0.034340


In [124]:
banten_test = ks_test(banten_normal, banten_lognormal, banten_gumbel, banten_lp3)
banten_test

Use Log Pearson 3


,Distribusi,Delta P,Nilai Kritis,Kesimpulan
0,Normal,0.209983,0.354,Diterima
1,Log Normal,0.144179,0.354,Diterima
2,Gumbel,0.156957,0.354,Diterima
3,Log Pearson 3,0.138160,0.354,Diterima


In [125]:
banten_design = design(banten_log, 'lp3')
banten_design

,T (Tahun),z,K,Rt (mm)
0,2,-0.000000,0.019495,90.076608
1,5,0.841621,0.846743,136.198030
2,10,1.281552,1.268321,168.142169
3,25,1.750686,1.709780,209.651414
4,50,2.053749,1.990553,241.235454
5,100,2.326348,2.240165,273.288212
6,200,2.575829,2.466185,305.970654
7,1000,3.090232,2.924950,384.819798


In [126]:
save_excel(
    [
        banten_df, banten_rain, banten_log, banten_normal, banten_lognormal,
        banten_gumbel, banten_lp3, banten_test, banten_design
    ], 'banten_design'
)

## Cisadane Watershed

In [127]:
cis_arit_2_df = process('cis_arit_2')
cis_arit_2_rain = rain(cis_arit_2_df)
cis_arit_2_log = log(cis_arit_2_df)

In [128]:
cis_arit_2_normal = normal(cis_arit_2_df, cis_arit_2_rain)
cis_arit_2_normal

,Rank,RR (mm),P_emp,k,P_teoritis,Delta P
0,1,140.800000,0.090909,1.870378,0.030716,0.060193
1,2,134.533333,0.181818,1.461172,0.071984,0.109834
2,3,119.833333,0.272727,0.501278,0.308088,0.035360
3,4,112.900000,0.363636,0.048539,0.480643,0.117007
4,5,108.366667,0.454545,-0.247483,0.597733,0.143187
5,6,105.833333,0.545455,-0.412907,0.660163,0.114708
6,7,105.433333,0.636364,-0.439026,0.669679,0.033315
7,8,101.866667,0.727273,-0.671926,0.749185,0.021912
8,9,97.133333,0.818182,-0.981007,0.836705,0.018524
9,10,94.866667,0.909091,-1.129018,0.870555,0.038536


In [129]:
cis_arit_2_lognormal = lognormal(cis_arit_2_df, cis_arit_2_log)
cis_arit_2_lognormal

,Rank,Log,P_emp,k,P_teoritis,Delta P
0,1,2.148603,0.090909,1.793938,0.036412,0.054498
1,2,2.128830,0.181818,1.446969,0.073953,0.107865
2,3,2.078578,0.272727,0.565150,0.285986,0.013259
3,4,2.052694,0.363636,0.110946,0.455829,0.092193
4,5,2.034896,0.454545,-0.201374,0.579797,0.125252
5,6,2.024622,0.545455,-0.381647,0.648639,0.103184
6,7,2.022978,0.636364,-0.410505,0.659282,0.022919
7,8,2.008032,0.727273,-0.672773,0.749454,0.022181
8,9,1.987368,0.818182,-1.035378,0.849754,0.031572
9,10,1.977114,0.909091,-1.215325,0.887879,0.021212


In [130]:
cis_arit_2_gumbel = gumbel(cis_arit_2_df, cis_arit_2_rain)
cis_arit_2_gumbel

,Rank,RR (mm),P_emp,k,P_teoritis,Delta P
0,1,140.800000,0.090909,1.870378,0.049715,0.041194
1,2,134.533333,0.181818,1.461172,0.082578,0.099240
2,3,119.833333,0.272727,0.501278,0.255615,0.017113
3,4,112.900000,0.363636,0.048539,0.409969,0.046333
4,5,108.366667,0.454545,-0.247483,0.537548,0.083003
5,6,105.833333,0.545455,-0.412907,0.614606,0.069152
6,7,105.433333,0.636364,-0.439026,0.626924,0.009440
7,8,101.866667,0.727273,-0.671926,0.735311,0.008039
8,9,97.133333,0.818182,-0.981007,0.861354,0.043173
9,10,94.866667,0.909091,-1.129018,0.908267,0.000824


In [131]:
cis_arit_2_lp3 = lp3(cis_arit_2_df, cis_arit_2_log)
cis_arit_2_lp3

,Rank,Log,P_emp,k,P_teoritis,Delta P
0,1,2.148603,0.090909,1.793938,0.052917,0.037992
1,2,2.128830,0.181818,1.446969,0.085795,0.096023
2,3,2.078578,0.272727,0.565150,0.255109,0.017619
3,4,2.052694,0.363636,0.110946,0.405181,0.041544
4,5,2.034896,0.454545,-0.201374,0.530423,0.075878
5,6,2.024622,0.545455,-0.381647,0.607131,0.061676
6,7,2.022978,0.636364,-0.410505,0.619490,0.016874
7,8,2.008032,0.727273,-0.672773,0.729685,0.002413
8,9,1.987368,0.818182,-1.035378,0.861846,0.043664
9,10,1.977114,0.909091,-1.215325,0.912133,0.003042


In [132]:
cis_arit_2_test = ks_test(cis_arit_2_normal, cis_arit_2_lognormal, cis_arit_2_gumbel, cis_arit_2_lp3)
cis_arit_2_test

Use Log Pearson 3


,Distribusi,Delta P,Nilai Kritis,Kesimpulan
0,Normal,0.143187,0.41,Diterima
1,Log Normal,0.125252,0.41,Diterima
2,Gumbel,0.099240,0.41,Diterima
3,Log Pearson 3,0.096023,0.41,Diterima


In [133]:
cis_arit_2_design = design(cis_arit_2_log, 'lp3')
cis_arit_2_design

,T (Tahun),z,K,Rt (mm)
0,2,-0.000000,-0.128367,109.409784
1,5,0.841621,0.779110,123.245371
2,10,1.281552,1.332651,132.530327
3,25,1.750686,1.987082,144.414030
4,50,2.053749,2.446659,153.390811
5,100,2.326348,2.885657,162.486204
6,200,2.575829,3.309298,171.774450
7,1000,3.090232,4.251103,194.370018


In [134]:
save_excel(
    [
        cis_arit_2_df, cis_arit_2_rain, cis_arit_2_log, cis_arit_2_normal, cis_arit_2_lognormal,
        cis_arit_2_gumbel, cis_arit_2_lp3, cis_arit_2_test, cis_arit_2_design
    ], 'cis_arit_2_design'
)